In [1]:
import time
import joblib
import requests
import random
import pandas as pd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options

In [2]:
data = joblib.load("../data/sp500_top100_v4.bin")
data.head()

,title,date,query,url,ticker,content,ps_content,ps_content_filter,sim
0,BRIEF-Apple Inc Says Not Allowing Entertainmen...,2020-03-15,Apple Inc.,https://www.reuters.com/article/idUSFWN2B61K2,AAPL,March 14 (Reuters) - Apple Inc: * APPLE INC SA...,,empty content,1.0
1,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSKBN20Z33J,AAPL,(Reuters) - Apple Inc has sealed multi-year li...,(Reuters) - Apple Inc has sealed multi-year li...,(Reuters) - Apple Inc has sealed multi-year li...,1.0
2,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSL4N2B54T2,AAPL,(Reuters) - Apple Inc has sealed multi-year li...,NaN,empty content,1.0
3,Chinese regulators remove 'Plague Inc' game fr...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSKCN20M043,AAPL,BEIJING/SHANGHAI (Reuters) - The video game “P...,The regulator did not respond to Reuters phone...,The regulator did not respond to Reuters phone...,1.0
4,UPDATE 1-Chinese regulators remove 'Plague Inc...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSL3N2AS0OO,AAPL,BEIJING/SHANGHAI (Reuters) - The video game “P...,NaN,empty content,1.0


In [3]:
USER_AGENTS = [
 "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
 "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
 "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
 "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
 "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
 "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
 "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
 "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
 "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
 "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
 "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
 "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
 "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
 "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]

In [4]:
def get_content(url):
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,fr;q=0.8,ro;q=0.7,ru;q=0.6,la;q=0.5,pt;q=0.4,de;q=0.3',
        'cache-control': 'max-age=0',
        'upgrade-insecure-requests': '1',
        'user-agent': '{}'.format(random.choice(USER_AGENTS))
    }
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    paragraph = []
    for element in soup.find_all("p"):
        paragraph.append("".join(element.find_all(text=True)))
    time.sleep(random.randint(2, 5))
    return "".join(paragraph[1:-2])

In [5]:
sp500_top20_company = [
    "AAPL", "MSFT", "AMZN", "FB", "GOOG", "JNJ", "BRK-B", "PG", "V", "JPM", 
    "UNH", "HD", "MA", "NVDA", "VZ", "PYPL", "NFLX", "ADBE", "PFE", "T"]

In [6]:
new_data = pd.DataFrame()
for ticker in sp500_top20_company:
    new_data = pd.concat([new_data, data[data["ticker"] == ticker]], axis=0)

new_data = new_data.iloc[:, 0:5]
new_data

,title,date,query,url,ticker
0,BRIEF-Apple Inc Says Not Allowing Entertainmen...,2020-03-15,Apple Inc.,https://www.reuters.com/article/idUSFWN2B61K2,AAPL
1,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSKBN20Z33J,AAPL
2,Apple signs multi-year deals with major music ...,2020-03-12,Apple Inc.,https://www.reuters.com/article/idUSL4N2B54T2,AAPL
3,Chinese regulators remove 'Plague Inc' game fr...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSKCN20M043,AAPL
4,UPDATE 1-Chinese regulators remove 'Plague Inc...,2020-02-28,Apple Inc.,https://www.reuters.com/article/idUSL3N2AS0OO,AAPL
...,...,...,...,...,...
126579,Deutsche Telekom CEO denies T-Mobile/Sprint de...,2019-12-10,AT%26T+Inc.,https://www.reuters.com/article/idUSKBN1YE2I3,T
126580,Nikkei rises as SoftBank Group soars on Sprint...,2020-02-12,AT%26T+Inc.,https://www.reuters.com/article/idUSL4N2AC1Q3,T
126581,REFILE-BRIEF-AT&T Expects 2020 Dividend Payout...,2020-06-18,AT%26T+Inc.,https://www.reuters.com/article/idUSFWN2DU0XL,T
126583,Japan Inc shifts to online press briefings as ...,2020-02-25,AT%26T+Inc.,https://www.reuters.com/article/idUSKCN20J17S,T


In [ ]:
tqdm.pandas()
for ticker in ["JNJ", "BRK-B", "PG", "V", "JPM"]:
    print("Get content of {}.".format(ticker))
    new_data["content"] = new_data[new_data["ticker"] == ticker]["url"].progress_apply(lambda row: get_content(row))
    joblib.dump(new_data, "../data/tickerdata/{}.bin".format(ticker), compress=5)

Get content of JNJ.


C:\Users\YangWang\AppData\Local\Continuum\anaconda3\envs\nlp\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
